In [5]:

import gzip
import json
import re
import requests

# —————————————————————————
# 1. 「イギリス」記事から基礎情報テンプレートを取得
# —————————————————————————
path = 'jawiki-country.json.gz'
with gzip.open(path, 'rt', encoding='utf-8') as f:
    text = None
    for line in f:
        dic = json.loads(line)
        if dic['title'] == 'イギリス':
            text = dic['text']
            break
if text is None:
    raise RuntimeError('「イギリス」記事が見つかりませんでした。')

tpl_re = re.compile(r'^\{\{基礎情報.+?$(.*?)^\}\}$',
                    re.MULTILINE | re.DOTALL)
m = tpl_re.search(text)
tpl_body = m.group(1)

fields = dict(re.findall(r'^\|\s*(.+?)\s*=\s*(.+)$',
                         tpl_body, flags=re.MULTILINE))

# —————————————————————————
# 2. 国旗画像ファイル名を取得
#    （例: 'Flag of the United Kingdom.svg'）
# —————————————————————————
# テンプレート上のキーは環境によって「国旗画像」や「国旗画像ファイル」など異なる場合があります
flag_field = '国旗画像'
filename = fields.get(flag_field)
if not filename:
    raise RuntimeError(f'テンプレートに「{flag_field}」が見つかりません。')

# 先頭に 'File:' がない場合は付与
if not filename.startswith(('File:', 'ファイル:')):
    filename = 'File:' + filename

# —————————————————————————
# 3. MediaWiki API にリクエストして URL を取得
# —————————————————————————
API_ENDPOINT = 'https://ja.wikipedia.org/w/api.php'
params = {
    'action': 'query',
    'format': 'json',
    'titles': filename,
    'prop': 'imageinfo',
    'iiprop': 'url'
}

resp = requests.get(API_ENDPOINT, params=params)
resp.raise_for_status()
data = resp.json()

# ページオブジェクトを取り出し、imageinfo の URL を抽出
pages = data['query']['pages']
page = next(iter(pages.values()))
imageinfo = page.get('imageinfo')
if not imageinfo:
    raise RuntimeError('imageinfo が返ってきませんでした。')

image_url = imageinfo[0]['url']
print('国旗画像 URL:', image_url)


国旗画像 URL: https://upload.wikimedia.org/wikipedia/commons/8/83/Flag_of_the_United_Kingdom_%283-5%29.svg
